# Notes for deployment

### A. Below Input files are read by this notebook, they need to be deployed to the AWS first:

1. "core-data_recipe.csv": recipe detail data (recipe_id, direction, ingredient list, nutrition fact, etc.)
Used in Recipe similarity calculation and recommendation. 
Link: https://github.com/wliu24/Capstone_Fall2020_Nutrition/blob/master/final_data_file/core-data_recipe.csv.zip

2. "recipesall.csv": recipe detail with more granular ingredient data
Used in ingredient similarity calculation and subsitition recommendation. 
Link: https://github.com/wliu24/Capstone_Fall2020_Nutrition/blob/master/final_data_file/recipesall.csv

3. "a_userinput.json": user input via the App (App output) regarding pregnancy stage, ingredient and nutrients preference, etc.
Link: https://github.com/wliu24/Capstone_Fall2020_Nutrition/blob/master/sample_JSON_file_new/a_userinput.json

4. "Nutrition_intake_reference.csv" : nutrient intake recommendation benchmark data, used to generate heatmap and flag
https://github.com/wliu24/Capstone_Fall2020_Nutrition/blob/master/final_data_file/Nutrition_intake_reference.csv

5. "Nutrient_level_nutrition_fact.csv":  


### B. Notebook Output is a Datafame, including:

1. Five Recommended recipe's recipe ID 
   And the details for the Five recommended recipes: direction, ingredient list, time, etc, for App display purpose
2. App Heatmap data:  Amount, %of the recommended intake amount, deficiency flag, for all nutrients to construct the Heatmap on the App
3. Ingredient subsitution list

### C. As Jennifer Suggested, a JSON file will be built up within Lambda function based on Notebook's output along with the above data files, for the App to read.
This target JSON could be found in:
https://github.com/wliu24/Capstone_Fall2020_Nutrition/blob/master/sample_JSON_file_new/b_recipelist_heatmap_recommendation_horizontal.json

1. Note1: The recipe_id should be join with file #1 above to get recipe detail for App display. The csv file is very straightforward. Or you could just solely use the Notebook output Dataframe to contrusct this JSON without join other flat files.

2. Note2: Any suggestion on the JSON file structure change could be discussed with Suzy directly to make front end code change; also make sure the upload the new JSON file structure to the same gitbut folder. 

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import collections
import json
import gensim 
import ast
from gensim.models import word2vec, phrases
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_numeric,\
                    strip_non_alphanum, strip_multiple_whitespaces, strip_short
from textblob import TextBlob, Word

import re
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
%matplotlib inline


from strsimpy.cosine import Cosine

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [3]:
recipe = pd.DataFrame()
for chunk in pd.read_csv('raw-data_recipe.csv',  chunksize=1000):
    recipe = pd.concat([recipe, chunk], ignore_index=True)
    
recipe = recipe.drop(columns=['image_url', 'reviews'])
recipe['nutritions'] = recipe['nutritions'].replace("\'", "\"").apply(ast.literal_eval)
recipe.head()

,recipe_id,recipe_name,aver_rate,review_nums,ingredients,cooking_directions,nutritions
0,222388,Homemade Bacon,5.000000,3,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,"{'niacin': {'hasCompleteData': False, 'name': ..."
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,29,sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{'niacin': {'hasCompleteData': False, 'name': ..."
2,218939,Foolproof Rosemary Chicken Wings,4.571429,12,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{'niacin': {'hasCompleteData': True, 'name': '..."
3,87211,Chicken Pesto Paninis,4.625000,163,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{'niacin': {'hasCompleteData': True, 'name': '..."
4,245714,Potato Bacon Pizza,4.500000,2,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{'niacin': {'hasCompleteData': True, 'name': '..."


In [4]:
sampledf1 = pd.DataFrame(recipe['nutritions'].values.tolist()).applymap(lambda x: x.get('amount', np.nan) \
                        if isinstance(x, dict) else np.nan)

In [5]:
# 'iron' mg, 'calcium' mg, 'folate' mcg, 'protein' g, 'vitaminA' iu, 'vitaminB6' mg, 'vitaminC' mg

new_recipe_file = pd.concat([recipe, sampledf1], axis=1)
df = new_recipe_file[['recipe_id', 'recipe_name', 'ingredients', 'cooking_directions', 'nutritions','iron', 'calcium',
                       'folate', 'protein', 'vitaminA', 'vitaminB6', 'vitaminC']]
original_df = df
df.head()

,recipe_id,recipe_name,ingredients,cooking_directions,nutritions,iron,calcium,folate,protein,vitaminA,vitaminB6,vitaminC
0,222388,Homemade Bacon,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,"{'niacin': {'hasCompleteData': False, 'name': ...",1.240848,11.18365,2.109131,21.002540,474.20730,0.232980,0.776127
1,240488,"Pork Loin, Apples, and Sauerkraut",sauerkraut drained^Granny Smith apples sliced^...,{'directions': u'Prep\n15 m\nCook\n2 h 30 m\nR...,"{'niacin': {'hasCompleteData': False, 'name': ...",6.622245,135.45380,83.739250,36.398780,73.17785,1.328631,52.768480
2,218939,Foolproof Rosemary Chicken Wings,chicken wings^sprigs rosemary^head garlic^oliv...,"{'directions': u""Prep\n20 m\nCook\n40 m\nReady...","{'niacin': {'hasCompleteData': True, 'name': '...",1.704567,60.08832,6.907802,23.912650,359.36400,0.553800,5.307448
3,87211,Chicken Pesto Paninis,focaccia bread quartered^prepared basil pesto^...,{'directions': u'Prep\n15 m\nCook\n5 m\nReady ...,"{'niacin': {'hasCompleteData': True, 'name': '...",5.011620,528.46170,234.213700,32.375370,604.75370,0.273496,18.015020
4,245714,Potato Bacon Pizza,red potatoes^strips bacon^Sauce:^heavy whippin...,{'directions': u'Prep\n20 m\nCook\n45 m\nReady...,"{'niacin': {'hasCompleteData': True, 'name': '...",1.024803,132.22650,49.481310,7.059566,168.32450,0.055718,0.905797


In [6]:
df.replace("[^a-zA-Z ]",", ",regex=True, inplace=True)

In [7]:
df = df.drop(columns = ['cooking_directions', 'nutritions'])

In [8]:
df['details'] = df['recipe_name'].str.lower() + ', '+ df['ingredients'].str.lower()
df.head(1)

,recipe_id,recipe_name,ingredients,iron,calcium,folate,protein,vitaminA,vitaminB6,vitaminC,details
0,222388,Homemade Bacon,"pork belly, smoked paprika, kosher salt, groun...",1.240848,11.18365,2.109131,21.00254,474.2073,0.23298,0.776127,"homemade bacon, pork belly, smoked paprika, ko..."


In [9]:
meat = ['beef', 'veal', 'pork', 'chicken', 'turkey', 'salmon', 'tuna', 'shrimp']
df['Meat'] = df['details'].str.contains('|'.join(meat))

In [10]:
spicy = ['chili', 'sriracha', 'spicy', 'jalapeno', 'scechuan']
df['Spicy'] = df['details'].str.contains('|'.join(spicy))

In [77]:
# Food Allergen
#https://www.fda.gov/food/buy-store-serve-safe-food/what-you-need-know-about-food-allergies
#Milk
#Eggs
#Fish (e.g., bass, flounder, cod)
#Crustacean shellfish (e.g., crab, lobster, shrimp)
#Tree nuts (e.g., almonds, walnuts, pecans)
#Peanuts
#Wheat
#Soybean

In [11]:
df['Soybean'] = df['details'].str.contains('soy')
df['Peanuts'] = df['details'].str.contains('peanut')

milk = ['milk', 'butter', 'yogurt', 'cream', 'cheese', 'gelato', 'half-and-half']
df['Milk'] = df['details'].str.contains('|'.join(milk))

egg = ['marshmallow', 'mayonnaise', 'meringue', 'frostings', 'custard', 'gelato', 'pretzel']
df['Eggs'] = df['details'].str.contains('|'.join(milk))

fish = ['bass', 'flounder', 'cod']
df['Fish'] = df['details'].str.contains('|'.join(fish))

shellfish = ['crab', 'lobster', 'shrimp']
df['Shell_fish'] = df['details'].str.contains('|'.join(shellfish))

treenuts = ['almond', 'walnut', 'pecan']
df['Tree_nuts'] = df['details'].str.contains('|'.join(treenuts))

wheat = ['bread', 'cake', 'pasta', 'farina', 'starch', 'soy sauce']
df['Wheat'] = df['details'].str.contains('|'.join(wheat))



In [12]:
df.head(1)

,recipe_id,recipe_name,ingredients,iron,calcium,folate,protein,vitaminA,vitaminB6,vitaminC,...,Meat,Spicy,Soybean,Peanuts,Milk,Eggs,Fish,Shell_fish,Tree_nuts,Wheat
0,222388,Homemade Bacon,"pork belly, smoked paprika, kosher salt, groun...",1.240848,11.18365,2.109131,21.00254,474.2073,0.23298,0.776127,...,True,False,False,False,False,False,False,False,False,False


In [11]:
#df.shape

In [12]:
#df_veggie = df.loc[df['Meat'] == False]

In [13]:
#df_veggie.head()

In [14]:
#details=[]
#for row in range(0,len(df.index)):
#    details.append(df.iloc[row,3])

In [15]:
#details

In [16]:
# input json  
# 'a_userinput.json' = {
#    "uuid": 1234,
#    "user_id": 1234,
#    "request_id": 1,
#    "request_time": "9/28/2020",
#    "user input": {
#        "motherhood_stage": "pregnancy first trimester",
#        "ingredient_list": ["chicken", "tomato"],
#        "nutrition_focus": ["full-eval","iron"],
#        "if_vegetierian": true,
#        "ingredient_exclusion": ["onion","cilantro","cheese"]
#    }
# }

# Read a_userinput.json

In [13]:
import pandas as pd
df_json_a = pd.read_json (r'a_userinput_2.json')
df_json_a

,uuid,user_id,request_id,request_time,user input
eggs,1234,1234,1,2020-09-28,False
excludeIngredients,1234,1234,1,2020-09-28,[beef]
fish,1234,1234,1,2020-09-28,False
includeIngredients,1234,1234,1,2020-09-28,"[chicken, tomato, potato]"
milk,1234,1234,1,2020-09-28,False
nonspicy,1234,1234,1,2020-09-28,False
nutritionPriority,1234,1234,1,2020-09-28,Overall
peanuts,1234,1234,1,2020-09-28,False
shellfish,1234,1234,1,2020-09-28,True
soybean,1234,1234,1,2020-09-28,False


In [14]:
json_input = df_json_a.iloc[3,4]
#json_input

In [15]:
ingredients_input = ', '.join(df_json_a.iloc[3,4])
ingredients_input
#type(ingredients_input)

'chicken, tomato, potato'

In [25]:
ingredients_excl = ', '.join(df_json_a.iloc[1,4])
ingredients_excl

'beef'

In [17]:
stage = str(df_json_a.iloc[10,4])
stage

'Pregnant - First Trimester'

In [19]:
#json_input
#type(json_input)

In [20]:
ingredient_1 = json_input[0]
ingredient_2 = json_input[1]
ingredient_3 = json_input[2]

In [21]:
ingredient_1

'chicken'

In [40]:
ingredient_1 = 'salmon'
ingredient_input = 'salmon, broccoli, carrot'
ingredients_excl = ' '

df['input'] = 'salmon, broccoli, carrot'
df['ingredient_1'] = 'salmon'
df['ingredient_2'] = 'broccoli'
df['ingredient_3'] = 'carrot'
df['stage'] = stage

In [27]:
df.head(1)

,recipe_id,recipe_name,ingredients,iron,calcium,folate,protein,vitaminA,vitaminB6,vitaminC,...,Tree_nuts,Wheat,input,ingredient_1,ingredient_2,ingredient_3,stage,p0,p1,cosine_sim
0,222388,Homemade Bacon,"pork belly, smoked paprika, kosher salt, groun...",1.240848,11.18365,2.109131,21.00254,474.2073,0.23298,0.776127,...,False,False,"beef, mushroom, zucchini",beef,mushroom,zucchini,Pregnant - First Trimester,"{'be': 1, 'ee': 1, 'ef': 1, 'f,': 1, ', ': 2, ...","{'ho': 1, 'om': 1, 'me': 1, 'em': 1, 'ma': 1, ...",0.227798


In [41]:
cosine = Cosine(2)
df["p0"] = df["input"].apply(lambda s: cosine.get_profile(s)) 
df["p1"] = df["details"].apply(lambda s: cosine.get_profile(s)) 
df["cosine_sim"] = [cosine.similarity_profiles(p0,p1) for p0,p1 in zip(df["p0"],df["p1"])]

df_2 = df.drop(["p0", "p1"], axis=1)

In [42]:
df_3 = df_2[df_2['details'].str.contains('|'.join(ingredients_input))]
df_4 = df_3[df_3['details'].str.contains(ingredient_1)]
df_5 = df_4[df_4['details'].str.contains(ingredients_excl) == False]
#df_5 = df_4[df_4['details'].str.contains('|'.join(ingredients_excl)) == False]

In [83]:
#df_4.head()

In [84]:
df_3.shape

(49698, 27)

In [85]:
df_4.shape

(7915, 27)

In [86]:
df_5.shape

(7658, 27)

In [43]:
output = df_4.nlargest(10, 'cosine_sim')
output

,recipe_id,recipe_name,ingredients,iron,calcium,folate,protein,vitaminA,vitaminB6,vitaminC,...,Fish,Shell_fish,Tree_nuts,Wheat,input,ingredient_1,ingredient_2,ingredient_3,stage,cosine_sim
42523,236640,Salmon Casserole Wrap,"sliced potatoes, cream of broccoli soup, salmo...",5.391870,374.25300,96.90893,34.44583,905.89280,0.611760,34.82712,...,False,False,False,False,"salmon, broccoli, carrot",salmon,broccoli,carrot,Pregnant - First Trimester,0.607644
36066,26350,Moroccan Salmon Cakes with Garlic Mayonnaise,"GARLIC MAYONNAISE, , mayonnaise, garlic, papri...",3.663593,370.30680,133.38980,28.77129,8977.83300,0.383995,25.41989,...,False,False,False,True,"salmon, broccoli, carrot",salmon,broccoli,carrot,Pregnant - First Trimester,0.563472
42366,56708,Honey Coconut Salmon,"butter, honey, brown sugar, flaked coconut, fi...",0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,...,False,False,False,False,"salmon, broccoli, carrot",salmon,broccoli,carrot,Pregnant - First Trimester,0.560991
42423,240921,Barbeque Roasted Salmon,"salmon fillets, lemon juice, cooking spray, br...",2.627618,115.05140,20.42081,36.85527,1733.00300,0.951241,17.61468,...,False,False,False,False,"salmon, broccoli, carrot",salmon,broccoli,carrot,Pregnant - First Trimester,0.554706
32950,57063,Curry Salmon with Broccoli,"uncooked long grain white rice, water, broccol...",4.069732,144.78580,221.25520,35.89945,1398.11200,1.408569,139.64100,...,False,False,False,False,"salmon, broccoli, carrot",salmon,broccoli,carrot,Pregnant - First Trimester,0.547117
39805,178249,"Dad, s Candied Salmon","salmon fillet, dry mustard, brown sugar, lemon",1.588525,67.18000,26.03500,45.31200,356.60750,1.032770,13.18875,...,False,False,False,False,"salmon, broccoli, carrot",salmon,broccoli,carrot,Pregnant - First Trimester,0.531489
43001,236939,"Mediterranean, Twist Salmon","Salmon, , olive oil, fillets salmon, Sauce, , ...",1.277702,43.90179,21.95700,24.78120,1396.41000,0.269700,9.08625,...,False,False,False,False,"salmon, broccoli, carrot",salmon,broccoli,carrot,Pregnant - First Trimester,0.527168
31258,70739,Candied Salmon,"butter, garlic, salmon fillets, large onions, ...",1.327758,63.95834,40.24208,23.19880,276.33540,0.935919,5.70600,...,False,False,False,False,"salmon, broccoli, carrot",salmon,broccoli,carrot,Pregnant - First Trimester,0.526742
41064,21736,Pan Seared Salmon I,"fillets salmon, olive oil, capers, salt, groun...",0.828468,30.51428,45.62056,33.70266,94.18022,1.086087,17.14568,...,False,False,False,False,"salmon, broccoli, carrot",salmon,broccoli,carrot,Pregnant - First Trimester,0.526742
35723,158272,Delicious Salmon Pasta Salad,"farfalle pasta, heads broccoli separated into ...",2.935065,202.89080,172.90260,23.06613,4898.48200,0.389826,113.00740,...,False,False,False,True,"salmon, broccoli, carrot",salmon,broccoli,carrot,Pregnant - First Trimester,0.525668


In [31]:
df_intake = pd.read_csv('nutrition_ref.csv')
df_intake

,stage,iron_ref,calcium_ref,folate_ref,protein_ref,vitaminA_ref,vitaminC_ref,vitaminB6_ref
0,Pregnant - First Trimester,27,1000,600,70,770,85,1.9
1,Pregnant - Second Trimester,27,1000,600,80,770,85,1.9
2,Pregnant - Third Trimester,27,1000,600,90,770,85,1.9
3,Breastfeeding,9,1000,500,70,1300,120,2.0


In [44]:
output2 = pd.merge(output,df_intake,how="left",on=None,left_on='stage',right_on='stage',left_index=True,
    right_index=False, sort=True, suffixes=("_1", "_2"),copy=True,indicator=False,validate=None)
output2

,recipe_id,recipe_name,ingredients,iron,calcium,folate,protein,vitaminA,vitaminB6,vitaminC,...,ingredient_3,stage,cosine_sim,iron_ref,calcium_ref,folate_ref,protein_ref,vitaminA_ref,vitaminC_ref,vitaminB6_ref
0,236640,Salmon Casserole Wrap,"sliced potatoes, cream of broccoli soup, salmo...",5.391870,374.25300,96.90893,34.44583,905.89280,0.611760,34.82712,...,carrot,Pregnant - First Trimester,0.607644,27,1000,600,70,770,85,1.9
0,26350,Moroccan Salmon Cakes with Garlic Mayonnaise,"GARLIC MAYONNAISE, , mayonnaise, garlic, papri...",3.663593,370.30680,133.38980,28.77129,8977.83300,0.383995,25.41989,...,carrot,Pregnant - First Trimester,0.563472,27,1000,600,70,770,85,1.9
0,56708,Honey Coconut Salmon,"butter, honey, brown sugar, flaked coconut, fi...",0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,...,carrot,Pregnant - First Trimester,0.560991,27,1000,600,70,770,85,1.9
0,240921,Barbeque Roasted Salmon,"salmon fillets, lemon juice, cooking spray, br...",2.627618,115.05140,20.42081,36.85527,1733.00300,0.951241,17.61468,...,carrot,Pregnant - First Trimester,0.554706,27,1000,600,70,770,85,1.9
0,57063,Curry Salmon with Broccoli,"uncooked long grain white rice, water, broccol...",4.069732,144.78580,221.25520,35.89945,1398.11200,1.408569,139.64100,...,carrot,Pregnant - First Trimester,0.547117,27,1000,600,70,770,85,1.9
0,178249,"Dad, s Candied Salmon","salmon fillet, dry mustard, brown sugar, lemon",1.588525,67.18000,26.03500,45.31200,356.60750,1.032770,13.18875,...,carrot,Pregnant - First Trimester,0.531489,27,1000,600,70,770,85,1.9
0,236939,"Mediterranean, Twist Salmon","Salmon, , olive oil, fillets salmon, Sauce, , ...",1.277702,43.90179,21.95700,24.78120,1396.41000,0.269700,9.08625,...,carrot,Pregnant - First Trimester,0.527168,27,1000,600,70,770,85,1.9
0,70739,Candied Salmon,"butter, garlic, salmon fillets, large onions, ...",1.327758,63.95834,40.24208,23.19880,276.33540,0.935919,5.70600,...,carrot,Pregnant - First Trimester,0.526742,27,1000,600,70,770,85,1.9
0,21736,Pan Seared Salmon I,"fillets salmon, olive oil, capers, salt, groun...",0.828468,30.51428,45.62056,33.70266,94.18022,1.086087,17.14568,...,carrot,Pregnant - First Trimester,0.526742,27,1000,600,70,770,85,1.9
0,158272,Delicious Salmon Pasta Salad,"farfalle pasta, heads broccoli separated into ...",2.935065,202.89080,172.90260,23.06613,4898.48200,0.389826,113.00740,...,carrot,Pregnant - First Trimester,0.525668,27,1000,600,70,770,85,1.9


In [45]:
output2['iron_percentage'] = output2['iron']/output2['iron_ref']
output2['calcium_percentage'] = output2['calcium']/output2['calcium_ref']
output2['folate_percentage'] = output2['folate']/output2['folate_ref']
output2['protein_percentage'] = output2['protein']/output2['protein_ref']
output2['vitaminA_percentage'] = (output2['vitaminA']*0.3)/(output2['vitaminA_ref'])
output2['vitaminB6_percentage'] = output2['vitaminB6']/output2['vitaminB6_ref']
output2['vitaminC_percentage'] = output2['vitaminC']/output2['vitaminC_ref']

In [34]:
output2.head()

,recipe_id,recipe_name,ingredients,iron,calcium,folate,protein,vitaminA,vitaminB6,vitaminC,...,vitaminA_ref,vitaminC_ref,vitaminB6_ref,iron_percentage,calcium_percentage,folate_percentage,protein_percentage,vitaminA_percentage,vitaminB6_percentage,vitaminC_percentage
0,237566,Ground Beef Pinwheels with Mushroom Sauce,"Meat, , ground beef, onion, garlic, ground bla...",5.740789,365.43060,108.74150,25.90936,219.15390,0.262391,2.082650,...,770,85,1.9,0.212622,0.365431,0.181236,0.370134,0.085385,0.138101,0.024502
0,50757,"Beef and Mushroom Stroganoff, Aussie Style","butter, onion, minced garlic, fresh mushrooms ...",5.399721,96.34171,170.86750,31.73664,1227.23800,0.623371,3.886837,...,770,85,1.9,0.199990,0.096342,0.284779,0.453381,0.478145,0.328090,0.045727
0,25250,Easiest Mushroom Sauce,"beef broth, all, purpose flour, mushrooms dice...",0.818144,10.00350,17.88508,1.70863,277.90540,0.033411,1.804833,...,770,85,1.9,0.030302,0.010004,0.029808,0.024409,0.108275,0.017585,0.021233
0,258947,Mushroom Beef Burgers,"ground beef, mushrooms chopped, onion, garlic,...",2.214629,18.87981,14.67812,20.09050,13.71181,0.251904,2.077813,...,770,85,1.9,0.082023,0.018880,0.024464,0.287007,0.005342,0.132581,0.024445
0,237579,Mushroom Meatloaf with Glaze,"Meatloaf, , onion, finely chopped mushroom, mi...",3.345505,100.93560,43.48056,25.55650,353.14780,0.360315,6.628445,...,770,85,1.9,0.123908,0.100936,0.072468,0.365093,0.137590,0.189639,0.077982


In [64]:
#result = output[['recipe_id']]

In [46]:
result2 = output2[['recipe_id','recipe_name', 'ingredients', 'protein', 'protein_percentage', 'calcium', 'calcium_percentage', 'iron', 'iron_percentage', 'folate', 'folate_percentage',
                 'vitaminA', 'vitaminA_percentage', 'vitaminC', 'vitaminC_percentage',  'vitaminB6', 'vitaminB6_percentage', 'ingredient_1', 'ingredient_2', 'ingredient_3', 'cosine_sim','input']]
result2.head(1)

,recipe_id,recipe_name,ingredients,protein,protein_percentage,calcium,calcium_percentage,iron,iron_percentage,folate,...,vitaminA_percentage,vitaminC,vitaminC_percentage,vitaminB6,vitaminB6_percentage,ingredient_1,ingredient_2,ingredient_3,cosine_sim,input
0,236640,Salmon Casserole Wrap,"sliced potatoes, cream of broccoli soup, salmo...",34.44583,0.492083,374.253,0.374253,5.39187,0.199699,96.90893,...,0.352945,34.82712,0.409731,0.61176,0.321979,salmon,broccoli,carrot,0.607644,"salmon, broccoli, carrot"


In [100]:
#result2.rename(columns={'protein':'protein_g', 'calcium':'calcium_mg', 'iron':'iron_mg', 'folate':'folate_mcg', 'vitaminA':'vitaminA_i', 'vitaminB6':'vitaminB6_mg', 'vitaminC':'vitaminC_mg'})


In [47]:
result2['p_c_i_pct'] = result2[['protein_percentage','iron_percentage','calcium_percentage']].mean(axis=1)

In [48]:
result2['protein_deficiency'] = np.where(result2['protein_percentage']>=0.33, 0, 1)
result2['iron_deficiency'] = np.where(result2['iron_percentage']>=0.33, 0, 1)
result2['calcium_deficiency'] = np.where(result2['calcium_percentage']>=0.33, 0, 1)
result2['folate_deficiency'] = np.where(result2['folate_percentage']>=0.33, 0, 1)
result2['vitaminA_deficiency'] = np.where(result2['vitaminA_percentage']>=0.33, 0, 1)
result2['vitaminB6_deficiency'] = np.where(result2['vitaminB6_percentage']>=0.33, 0, 1)
result2['vitaminC_deficiency'] = np.where(result2['vitaminC_percentage']>=0.33, 0, 1)

In [49]:
recipe_list = result2.nlargest(3, 'p_c_i_pct')
recipe_list

,recipe_id,recipe_name,ingredients,protein,protein_percentage,calcium,calcium_percentage,iron,iron_percentage,folate,...,cosine_sim,input,p_c_i_pct,protein_deficiency,iron_deficiency,calcium_deficiency,folate_deficiency,vitaminA_deficiency,vitaminB6_deficiency,vitaminC_deficiency
0,236640,Salmon Casserole Wrap,"sliced potatoes, cream of broccoli soup, salmo...",34.44583,0.492083,374.2530,0.374253,5.391870,0.199699,96.90893,...,0.607644,"salmon, broccoli, carrot",0.355345,0,1,0,1,0,1,0
0,26350,Moroccan Salmon Cakes with Garlic Mayonnaise,"GARLIC MAYONNAISE, , mayonnaise, garlic, papri...",28.77129,0.411018,370.3068,0.370307,3.663593,0.135689,133.38980,...,0.563472,"salmon, broccoli, carrot",0.305671,0,1,0,1,0,1,1
0,57063,Curry Salmon with Broccoli,"uncooked long grain white rice, water, broccol...",35.89945,0.512849,144.7858,0.144786,4.069732,0.150731,221.25520,...,0.547117,"salmon, broccoli, carrot",0.269455,0,1,1,0,0,0,0


In [50]:
recipe_list[['input','recipe_name','ingredients','cosine_sim']]

,input,recipe_name,ingredients,cosine_sim
0,"salmon, broccoli, carrot",Salmon Casserole Wrap,"sliced potatoes, cream of broccoli soup, salmo...",0.607644
0,"salmon, broccoli, carrot",Moroccan Salmon Cakes with Garlic Mayonnaise,"GARLIC MAYONNAISE, , mayonnaise, garlic, papri...",0.563472
0,"salmon, broccoli, carrot",Curry Salmon with Broccoli,"uncooked long grain white rice, water, broccol...",0.547117
